In [16]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets.samples_generator import make_regression 

sizes = np.loadtxt('/home/dawid/git/machine_learning_for_students/flat.areas').reshape((26,1))
prices = np.loadtxt('/home/dawid/git/machine_learning_for_students/flat.prices').reshape((26,1))
samples_cnt = len(prices)
weight0 = 0.0
weight1 = 0.0
learning_rate = 0.0001
maxIteration = 100000

for i in range(maxIteration):
    weight0 = weight0
    weight1 = weight1
    
    #Zadanie1: 
    predicted_prices = []
    #uzupełnij listę predicted_prices tak, aby dla każdego metrazu budynku z listy 'sizes' wyznaczyć cenę tegoż
    #budynku przy użyciu aktualnych wag modelu liniowego
    
    #Zadanie2:
    #napisz kod, który iteracyjnie poprawiać będzie wagi tak, aby ostatecznie wyznaczyły prostą,
    #która najlepiej odwzoruje zależność metraż -> cena

plt.plot(sizes, prices, "x")
if len(predicted_prices) > 0:
    plt.plot(sizes, predicted_prices, "r-")
plt.title('Ceny mieszkan w zaleznosci od metrazu')
plt.xlabel('metraz (m^2)')
plt.ylabel('Cena mieskania (tys. zł)')
plt.show()
print("Wartości wag -> w0:", weight0, "w1:", weight1)

ValueError: x and y must have same first dimension